## Evaluation Phase

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys


from collections import namedtuple

%matplotlib inline
#
from evaluation import *
from approximator import *
from q_learning_nn import *

#import environment
sys.path.append(r'../virl')
import virl

In [2]:

#load test environment

for num_instance in range(10):
    env = virl.Epidemic(problem_id=num_instance, noisy=True, stochastic=True)

    n_episode = 100

    alpha = 0.0
    nn_config = [36, 36]
    BATCH_SIZE = 10
    BUFFER_SIZE = 10000

    d_states = env.observation_space.shape[0]
    n_actions = env.action_space.n

    nn_approximator =  NNFunctionApproximatorJointKeras(alpha, d_states, n_actions, nn_config)
    nn_approximator_target =  NNFunctionApproximatorJointKeras(alpha, d_states, n_actions, nn_config)

    stats = q_learning_nn(env, nn_approximator, func_approximator_target, use_normalization =True,
    max_se
    )